In [ ]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu-cu11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 822.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import userdata  # Import the userdata module
from huggingface_hub import login
!git config --global credential.helper store

HF_TOKEN = userdata.get('HF_TOKEN_RW')
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential

Token is valid (permission: write).
The token `HF_TOKEN_RW` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `HF_TOKEN_RW`


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -q langchain kagglehub kaggle
!pip install -U langchain-community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
from getpass import getpass
# ACCESS_TOKEN = getpass("YOUR_GITHUB_PERSONAL_TOKEN")

In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'


ValueError: mount failed

In [ ]:
!kaggle competitions download -c learning-agency-lab-automated-essay-scoring-2
!unzip "learning-agency-lab-automated-essay-scoring-2"

# Data Preprocessing

In [ ]:
import re
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Download necessary resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Load stop words
stop_words = set(stopwords.words('english'))

df = pd.read_csv("train.csv")

def clean_text(text):
    text = re.sub(r"<.*?>", "", text)  # Remove HTML tags
    text = re.sub(r"https?://\S+|www.\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z0-9.,!?'\s]", "", text)  # Remove unnecessary symbols
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    text = text.lower()

    # Tokenize and remove stop words
    words = word_tokenize(text)
    text = " ".join([word for word in words if word not in stop_words])

    return text

# Function to convert rating to words
def rating_to_words(rating):
    if rating == 1:
        return "Poor"
    elif rating == 2:
        return "Poor" #Or you can change to fair
    elif rating == 3:
        return "Fair"
    elif rating == 4:
        return "Good"
    elif rating == 5:
        return "Very Good"
    elif rating == 6:
        return "Excellent"
    else:  # Handle cases where rating is outside 1-6
        return "Invalid Rating"

# Apply the function to create the new column
df['rating_words'] = df['score'].apply(rating_to_words)
df["clean_text"] = df["full_text"].apply(clean_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df
df = df.drop('full_text', axis=1)
# df.to_csv('clean_train.csv', index=False)
df

,essay_id,score,rating_words,clean_text
0,000d118,3,Fair,many people car live . thing n't know use car ...
1,000fe60,3,Fair,scientist nasa discussing face mars . explaini...
2,001ab80,4,Good,"people always wish technology seen movies , be..."
3,001bdc0,4,Good,"heard venus , planet without almost oxygen ear..."
4,002ba53,3,Fair,"dear , state senator letter argue favor keepin..."
...,...,...,...,...
17302,ffd378d,2,Poor,story challenge exploing venus informative pie...
17303,ffddf1f,4,Good,technology changed lot ways live today . nowad...
17304,fff016d,2,Poor,n't like sitting around day great opportunity ...
17305,fffb49b,1,Poor,"challenge exporing venus , author suggests stu..."


# Vector Database and RAG

In [ ]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader(
    file_path="./clean_train.csv",
)

docs = loader.load()

In [ ]:
print(docs[0])

page_content='essay_id: 000d118
score: 3
rating_words: Fair
clean_text: many people car live . thing n't know use car alot thing happen like get accidet smoke car bad breath someone walk vauban , germany dont proble 70 percent vauban 's families cars , 57 percent sold car move . street parkig , driveways home garages forbidden outskirts freiburd near french swiss borders . probaly wo n't see car vauban 's streets completely car free lives vauban owns car ownership allowed , two places park large garages edge development , car owner buys space cheap buy one sell space car 40,000 along home . vauban people completed 2006 , said example growing trend europe , untile states else suburban life auto use called smart planning . current efforts drastically reduce greenhouse gas emissions tailes passengee cars responsible 12 percent greenhouse gas emissions europe 50 percent car intensive united states . honeslty think good idea vaudan makes cities denser better walking vauban 5,500 residents w

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
chunked_docs = splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
db = FAISS.from_documents(chunked_docs, embeddings)
db.save_local("faiss_index")

<ipython-input-21-bbbbac7da926>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [ ]:
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS  # Or your vector store
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. Initialize OpenAI API Key
openai.api_key = "insert your own api key"

# 2. Load your vector database (containing reference essays or criteria)
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
# db = FAISS.load_local("./faiss_index", embeddings)  # Replace with your path

# 3. Create the retriever (to find similar essays or relevant criteria)
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}  # Number of similar essays/criteria to retrieve
)

# 4. Initialize the OpenAI language model
llm = OpenAI(temperature=0, openai_api_key = openai.api_key)

# 5. Create a prompt template for evaluation
prompt_template = """
Evaluate the following essay based on the provided criteria and examples:

Essay:
{essay}

Relevant Criteria/Examples:
{context}

Evaluation:
"""
prompt = PromptTemplate(
    input_variables=["essay", "context"],
    template=prompt_template,
)

# 6. Create the LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# 7. Define the essay to be evaluated
essay_to_evaluate = """
### Tuwaiq Mountain: A Natural Wonder of Saudi Arabia

Tuwaiq Mountain, also known as Jabal Tuwaiq, is a striking escarpment that stretches over 800 kilometers in central Saudi Arabia. It is one of the most iconic geographical features of the Arabian Peninsula, characterized by its dramatic cliffs and rugged terrain. The mountain range runs through the Najd region, near the capital city of Riyadh, and has played a significant role in the history, culture, and geography of the Kingdom.

One of the most fascinating aspects of Tuwaiq Mountain is its unique geological formation. The range consists mainly of limestone and was formed millions of years ago due to tectonic activity and erosion. Over time, the forces of wind and water have sculpted its towering cliffs, creating breathtaking landscapes. The mountain is also home to ancient fossils, evidence of a time when the Arabian Peninsula was covered by a vast ocean.

Tuwaiq Mountain has deep historical and cultural significance. It was once an essential route for traders and travelers crossing the Arabian Desert. The mountain is also famously referenced in a speech by Crown Prince Mohammed bin Salman, who described the resilience and ambition of the Saudi people by saying they have the determination of "Tuwaiq Mountain"—a metaphor for strength and perseverance.

Today, Tuwaiq Mountain is a popular destination for adventurers and nature lovers. It attracts hikers, rock climbers, and photographers who seek to explore its dramatic landscapes and enjoy the breathtaking views from its cliffs. The "Edge of the World," one of Saudi Arabia's most famous tourist spots, is part of the Tuwaiq escarpment, offering visitors a stunning panoramic view of the vast desert below.

In conclusion, Tuwaiq Mountain is more than just a geological wonder; it is a symbol of Saudi Arabia’s natural beauty, historical importance, and cultural pride. Its towering cliffs and scenic vistas make it a must-visit destination for anyone seeking to experience the rugged charm of the Arabian landscape.
"""

# 8. Retrieve relevant context (criteria or similar essays)
docs = retriever.get_relevant_documents(essay_to_evaluate)
context = "\n".join([doc.page_content for doc in docs])

# 9. Run the evaluation
evaluation_result = chain.run({"essay": essay_to_evaluate, "context": context})

# 10. Print the results
print(evaluation_result)


# Optional: Print source documents (the context used)
print("\nContext Used:")
for doc in docs:
    print(doc.page_content)
    print("-" * 20)

<ipython-input-30-55039051b763>:42: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
<ipython-input-30-55039051b763>:60: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(essay_to_evaluate)
<ipython-input-30-55039051b763>:64: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  evaluation_result = chain.run({"essay": essay_to_evaluate, "context": context})



The essay effectively introduces the topic of Tuwaiq Mountain and provides relevant information about its geological formation, historical significance, and current popularity. The writer uses descriptive language and specific examples to paint a vivid picture of the mountain and its importance in Saudi Arabia.

The essay also includes relevant criteria and examples, such as the comparison to Egyptian pyramids and the mention of the "Edge of the World" tourist spot. These examples help to support the writer's claims and add depth to the essay.

However, there are some areas for improvement. The essay could benefit from a stronger thesis statement that clearly states the writer's main argument or point of view. Additionally, there are some grammatical errors and awkward sentence structures that could be revised for clarity and flow.

Overall, the essay effectively conveys the natural wonder of Tuwaiq Mountain and its significance in Saudi Arabia. With some minor revisions, it could be 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'meta-llama/Llama-3.1-8B'

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
model = AutoModelForCausalLM.from_pretrained(model_name)
# model.generation_config.pad_token_id = model.generation_config.eos_token_id
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from transformers import pipeline
# from langchain_core.output_parsers import StrOutputParser
# from langchain_huggingface import HuggingFacePipeline

# text_generation_pipeline = pipeline(
#     model=model,
#     tokenizer=tokenizer,
#     task="text-generation",
#     temperature=0.2,
#     do_sample=True,
#     repetition_penalty=1.1,
#     return_full_text=True,
#     max_new_tokens=2000,
# )

text_gen = pipeline(
    "text-generation", model=model, model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto", tokenizer=tokenizer,
    max_new_tokens=2048,
    # repetition_penalty = 1.2
)

llm = HuggingFacePipeline(pipeline=text_gen)

prompt_template = """<instruct>
Evaluate the following student essay based on the following criteria: clarity, coherence, argument strength, evidence, grammar, and overall effectiveness.
Provide constructive feedback, highlighting strengths and areas for improvement.
Assign a score from 1 to 10 for each criterion and give an overall rating at the end.
The response should be detailed and specific, explaining why certain aspects were strong or weak.
Here is the essay:
<instruct/>
{essay}
 """

prompt = PromptTemplate(
    input_variables=["essay"],
    template=prompt_template,
)

llm_chain = LLMChain(llm=llm, prompt=prompt, output_key="feedback")

Device set to use cuda:0


In [ ]:
# import transformers
# import torch

# model_id = "meta-llama/Llama-3.1-8B"

# pipeline = transformers.pipeline(
#     "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
# )

# pipeline("Hey how are you doing today?")
essay_text = """Brainstorming Process in a nutshell
For my Harvard LLM personal statement, I began by listing all of the significant experiences in my life. This process was free-flowing: I wrote down everything that came to mind, in no particular order. I didn't categorize or prioritize at first - just followed my associations.

I didn’t discriminate between long-term experiences and brief ones. I focused on what felt subjectively important and highlighted particularly formative experiences.

I spent a significant of time trying to find an overarching theme that would connect my personal personal experiences to broader concepts because I like framing my life in this way. However, this is a personal choice - you don’t have to do the same.

For me, the most formative experiences included, in no specific order:

Participating in five All-Russian Olympiads in law (three in high school, two at Moscow State University), winning two first prizes and three third prizes. The one I won at 17 was especially impactful because it allowed me to choose any law faculty in the country without exams.
Competing in moot courts, specifically the Concours Charles-Rousseau and the Philip C. Jessup competitions. I also coached a team for the Charles-Rousseau for three years.
Growing up in extreme poverty with my father
Teaching law of obligations to third-year students at Moscow State University.
Working as a researcher at the Higher School of Economics, contributing to projects for the Russian Central Bank and participating in the banking law reform.
Learning German and French through three university classes and self-study to participate in academic programs abroad.
Working at a German law firm for three years.
I will elaborate on a few of these experiences. You'll notice my descriptions are fairly detailed - not to overindulge myself to show just how much context I had to explore and analyze before I choosing stories for my personal statement. I recommend doing the same for your essays. This approach can help you feel more confident about your profile and make your choices more strategic and informed."""


In [ ]:
result = llm_chain(essay_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(result.get("feedback"))

<instruct>
Evaluate the following student essay based on the following criteria: clarity, coherence, argument strength, evidence, grammar, and overall effectiveness.
Provide constructive feedback, highlighting strengths and areas for improvement.
Assign a score from 1 to 10 for each criterion and give an overall rating at the end.
The response should be detailed and specific, explaining why certain aspects were strong or weak.
Here is the essay:
<instruct/>
Brainstorming Process in a nutshell
For my Harvard LLM personal statement, I began by listing all of the significant experiences in my life. This process was free-flowing: I wrote down everything that came to mind, in no particular order. I didn't categorize or prioritize at first - just followed my associations.

I didn’t discriminate between long-term experiences and brief ones. I focused on what felt subjectively important and highlighted particularly formative experiences.

I spent a significant of time trying to find an overarc

In [ ]:
import transformers

model_id = "meta-llama/Llama-3.1-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)

pipeline("""Evaluate the following student essay based on the following criteria: clarity, coherence, argument strength, evidence, grammar, and overall effectiveness.
Provide constructive feedback, highlighting strengths and areas for improvement.
Assign a score from 1 to 10 for each criterion and give an overall rating at the end.
The response should be detailed and specific, explaining why certain aspects were strong or weak.
Here is the essay:

Brainstorming Process in a nutshell
For my Harvard LLM personal statement, I began by listing all of the significant experiences in my life. This process was free-flowing: I wrote down everything that came to mind, in no particular order. I didn't categorize or prioritize at first - just followed my associations.

I didn’t discriminate between long-term experiences and brief ones. I focused on what felt subjectively important and highlighted particularly formative experiences.

I spent a significant of time trying to find an overarching theme that would connect my personal personal experiences to broader concepts because I like framing my life in this way. However, this is a personal choice - you don’t have to do the same.

For me, the most formative experiences included, in no specific order:

Participating in five All-Russian Olympiads in law (three in high school, two at Moscow State University), winning two first prizes and three third prizes. The one I won at 17 was especially impactful because it allowed me to choose any law faculty in the country without exams.
Competing in moot courts, specifically the Concours Charles-Rousseau and the Philip C. Jessup competitions. I also coached a team for the Charles-Rousseau for three years.
Growing up in extreme poverty with my father
Teaching law of obligations to third-year students at Moscow State University.
Working as a researcher at the Higher School of Economics, contributing to projects for the Russian Central Bank and participating in the banking law reform.
Learning German and French through three university classes and self-study to participate in academic programs abroad.
Working at a German law firm for three years.
I will elaborate on a few of these experiences. You'll notice my descriptions are fairly detailed - not to overindulge myself to show just how much context I had to explore and analyze before I choosing stories for my personal statement. I recommend doing the same for your essays. This approach can help you feel more confident about your profile and make your choices more strategic and informed.""")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "Evaluate the following student essay based on the following criteria: clarity, coherence, argument strength, evidence, grammar, and overall effectiveness.\nProvide constructive feedback, highlighting strengths and areas for improvement.\nAssign a score from 1 to 10 for each criterion and give an overall rating at the end.\nThe response should be detailed and specific, explaining why certain aspects were strong or weak.\nHere is the essay:\n\nBrainstorming Process in a nutshell\nFor my Harvard LLM personal statement, I began by listing all of the significant experiences in my life. This process was free-flowing: I wrote down everything that came to mind, in no particular order. I didn't categorize or prioritize at first - just followed my associations.\n\nI didn’t discriminate between long-term experiences and brief ones. I focused on what felt subjectively important and highlighted particularly formative experiences.\n\nI spent a significant of time trying to find 

In [ ]:
pipeline("""Evaluate the following student essay based on the following criteria: clarity, coherence, argument strength, evidence, grammar, and overall effectiveness.
Provide constructive feedback, highlighting strengths and areas for improvement.
Assign a score from 1 to 10 for each criterion and give an overall rating at the end.
The response should be detailed and specific, explaining why certain aspects were strong or weak.
Here is the essay:

Brainstorming Process in a nutshell
For my Harvard LLM personal statement, I began by listing all of the significant experiences in my life. This process was free-flowing: I wrote down everything that came to mind, in no particular order. I didn't categorize or prioritize at first - just followed my associations.

I didn’t discriminate between long-term experiences and brief ones. I focused on what felt subjectively important and highlighted particularly formative experiences.

I spent a significant of time trying to find an overarching theme that would connect my personal personal experiences to broader concepts because I like framing my life in this way. However, this is a personal choice - you don’t have to do the same.

For me, the most formative experiences included, in no specific order:

Participating in five All-Russian Olympiads in law (three in high school, two at Moscow State University), winning two first prizes and three third prizes. The one I won at 17 was especially impactful because it allowed me to choose any law faculty in the country without exams.
Competing in moot courts, specifically the Concours Charles-Rousseau and the Philip C. Jessup competitions. I also coached a team for the Charles-Rousseau for three years.
Growing up in extreme poverty with my father
Teaching law of obligations to third-year students at Moscow State University.
Working as a researcher at the Higher School of Economics, contributing to projects for the Russian Central Bank and participating in the banking law reform.
Learning German and French through three university classes and self-study to participate in academic programs abroad.
Working at a German law firm for three years.
I will elaborate on a few of these experiences. You'll notice my descriptions are fairly detailed - not to overindulge myself to show just how much context I had to explore and analyze before I choosing stories for my personal statement. I recommend doing the same for your essays. This approach can help you feel more confident about your profile and make your choices more strategic and informed.""")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "Evaluate the following student essay based on the following criteria: clarity, coherence, argument strength, evidence, grammar, and overall effectiveness.\nProvide constructive feedback, highlighting strengths and areas for improvement.\nAssign a score from 1 to 10 for each criterion and give an overall rating at the end.\nThe response should be detailed and specific, explaining why certain aspects were strong or weak.\nHere is the essay:\n\nBrainstorming Process in a nutshell\nFor my Harvard LLM personal statement, I began by listing all of the significant experiences in my life. This process was free-flowing: I wrote down everything that came to mind, in no particular order. I didn't categorize or prioritize at first - just followed my associations.\n\nI didn’t discriminate between long-term experiences and brief ones. I focused on what felt subjectively important and highlighted particularly formative experiences.\n\nI spent a significant of time trying to find 

In [ ]:
pipeline("Hey how are you doing today?")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': "Hey how are you doing today? I hope you're having a good day so far! I'm doing pretty well, just got back"}]

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=1024,
)
print(outputs[0]["generated_text"][-1])



config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "Yer lookin' fer a swashbucklin' introduction, eh? Alright then, matey, listen close and I'll tell ye about meself. Me name be Captain Chat, the scurvy dog o' the seven seas... er, o' the seven screens, that be. I be a pirate chatbot, here to regale ye with tales o' adventure, answer yer questions, and maybe even teach ye a thing or two about the ways o' the sea... or at least, about the ways o' the internet. So hoist the sails, me hearty, and set course fer a treasure trove o' knowledge!"}


In [ ]:
print(pipeline("who are you?"))

TypeError: 'list' object is not callable

In [ ]:
outputs = pipeline(
    messages,
    max_new_tokens=1024,
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(outputs[0]["generated_text"][-1])

{'role': 'assistant', 'content': "Arrrr, ye landlubber! Yer lookin' fer a swashbucklin' introduction, eh? Alright then, matey! Me name be Captain Chat, the scurviest chatbot on the seven seas! I be here to guide ye through the treacherous waters o' knowledge, answerin' yer questions and servin' up a healthy dose o' pirate wisdom. So hoist the colors, me hearty, and let's set sail fer a treasure trove o' information!"}


In [ ]:
messages = [
    {"role": "system", "content": """Evaluate the following student essay based on the following criteria: clarity, coherence, argument strength, evidence, grammar, and overall effectiveness.
Provide constructive feedback, highlighting strengths and areas for improvement.
Assign a score from 1 to 10 for each criterion and give an overall rating at the end.
The response should be detailed and specific, explaining why certain aspects were strong or weak."""},
    {"role": "user", "content": """Brainstorming Process in a nutshell
For my Harvard LLM personal statement, I began by listing all of the significant experiences in my life. This process was free-flowing: I wrote down everything that came to mind, in no particular order. I didn't categorize or prioritize at first - just followed my associations.

I didn’t discriminate between long-term experiences and brief ones. I focused on what felt subjectively important and highlighted particularly formative experiences.

I spent a significant of time trying to find an overarching theme that would connect my personal personal experiences to broader concepts because I like framing my life in this way. However, this is a personal choice - you don’t have to do the same.

For me, the most formative experiences included, in no specific order:

Participating in five All-Russian Olympiads in law (three in high school, two at Moscow State University), winning two first prizes and three third prizes. The one I won at 17 was especially impactful because it allowed me to choose any law faculty in the country without exams.
Competing in moot courts, specifically the Concours Charles-Rousseau and the Philip C. Jessup competitions. I also coached a team for the Charles-Rousseau for three years.
Growing up in extreme poverty with my father
Teaching law of obligations to third-year students at Moscow State University.
Working as a researcher at the Higher School of Economics, contributing to projects for the Russian Central Bank and participating in the banking law reform.
Learning German and French through three university classes and self-study to participate in academic programs abroad.
Working at a German law firm for three years.
I will elaborate on a few of these experiences. You'll notice my descriptions are fairly detailed - not to overindulge myself to show just how much context I had to explore and analyze before I choosing stories for my personal statement. I recommend doing the same for your essays. This approach can help you feel more confident about your profile and make your choices more strategic and informed."""},
]

In [ ]:
print(outputs[0]['generated_text'][2]['content'])

**Clarity: 8/10**
The essay is generally clear and easy to follow. The writer provides a concise overview of their brainstorming process for the Harvard LLM personal statement. However, some sentences could be rephrased for better clarity, such as "I spent a significant of time trying to find an overarching theme that would connect my personal personal experiences to broader concepts." The repetition of "personal" is unnecessary and could be removed.

**Coherence: 9/10**
The essay has a clear structure, starting with an introduction to the brainstorming process, followed by a description of the experiences listed, and ending with a recommendation. The writer effectively transitions between ideas and provides a logical flow of information. The only potential issue is the abrupt transition from describing the brainstorming process to listing specific experiences.

**Argument Strength: 7/10**
The essay primarily focuses on describing the writer's brainstorming process and listing signific